In [ ]:
Лаба_2:
Компрессия LLM.
Модель: Qwen3-8B https://huggingface.co/Qwen/Qwen3-8B (модель брать
Датасет для бенчмарка: MMLU https://github.com/hendrycks/test (как вариант можно глянуть сюда как делать замеры)

Что требуется сделать?
Этап_1:
Цель данного этапа в том, чтобы получить максимально возможную компрессию при наименьшем падении метрики.
Дообучение моделей в привычном понимании запрещено, использовать Speculative Decoding тоже нельзя.
PEFT по типу P-tuning, PV-tuning тоже нельзя использовать. То есть все методы, которые в целом считают градиент.
Пост-трейнинг разрешен.

Что от вас ожидается:
1. Создаете гит репозиторий.
2. Веса модели кидаете на гугл или другой диск или любое удобное хранилище
3. Должен быть один скрипт,который воспроизведет ваши результаты компрессии на исходной модели. Допустим compress.py
4. Должен быть скрипт, который отдельно подгрузит уже сохраненную сжатую модель и замерит: 
  - количество параметров до и после компрессии, чтобы понимать ratio
  - качество на бенчмарке MMLU до и после, чтобы понимать какая деградация у модели

Вопросы по железу: Вам без разницы какое железо используется, какое GPU. Так что затачиваться под какое-то специально смысла нет.

Как будет строиться оценка:
(5-4): равное качество с baseline. Compression ratio: x1.2, metric drop: 4.5 пунктов по всему бенчмарку в среднем.
(7-6): решение должно быть лучше baseline. Min compression ratio: x1.4, metric drop не хуже чем в baseline 
(10-8): top-30% пар, которые оказались лучше всех.
Как я буду сравнивать ваши работы друг с другом?
Формула:
Score = Compression_ratio / (1 + Performance_drop)
Compression_ratio = Original_size / Compressed_size
Performance_drop = (Original_metric - Compressed_metric) / Original_metric

Пример:
Исходная модель: 1000Мб
Metric: 90% (выше лучше)
Студент_1: модель: 250мб metric: 70%
Студент_2: модель: 400мб metric: 78%

Студент_1:
Compression_ratio = 1000/250 = 4
Performance_drop = (90 - 70) / 90 = 0.2
Score = 4 / (1 + 0.2) = 3.3

Студент_2:
Compression_ratio = 1000/400 = 2.5
Performance_drop = (90 - 78) / 90 = 0.13
Score = 2.5 / (1 + 0.13) = 2.21

Студент_1 занимает место выше Студента_2

In [6]:
import os
os.environ['HF_TOKEN'] = ''

import os
from openai import OpenAI

In [6]:
# !pip install transformers==4.57.1 sentencepiece==0.2.1 datasets==4.0.0 accelerate==1.11.0 Ninja==1.13.0 triton==3.4.0

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import json
from tqdm import tqdm
from typing import Dict, Tuple, List
from pathlib import Path
import math
import argparse
import zipfile
import shutil
from pathlib import Path

In [ ]:
# АДДИТИВНЫЙ КВАНТАЙЗЕР (AdditiveQuantizer)

class AdditiveQuantizer(nn.Module):
    
    def __init__(self, 
                 n_books: int = 4, 
                 codebook_size: int = 256, 
                 codebook_dim: int = 128, 
                 learning_rate: float = 0.01,
                 device: str = "cuda"):
        super().__init__()
        
        self.n_books = n_books
        self.codebook_size = codebook_size
        self.codebook_dim = codebook_dim
        self.lr = learning_rate
        self.device = device
        
        # Инициализация кодовых книг
        self.codebooks = nn.ParameterList()
        for _ in range(n_books):
            codebook = torch.randn(codebook_size, codebook_dim, 
                                   device=device, dtype=torch.float32) * 0.01
            self.codebooks.append(nn.Parameter(codebook))
        
        
    @torch.no_grad()
    def _find_best_indices(self, weight_split: torch.Tensor, codebook: torch.Tensor) -> torch.Tensor:
        """Находит ближайший кодвектор для каждого под-вектора."""
        distances = torch.cdist(weight_split, codebook)
        indices = torch.argmin(distances, dim=1)
        return indices
    
    def quantize(self, 
                 weight: torch.Tensor,
                 num_iterations: int = 100,
                 commitment_weight: float = 0.25) -> Tuple[torch.Tensor, Dict]:
        
        rows, cols = weight.shape
        original_dtype = weight.dtype
        weight_f32 = weight.to(torch.float32)
        
        # Padding и Splitting
        pad_size = 0
        if cols % self.codebook_dim != 0:
            pad_size = self.codebook_dim - (cols % self.codebook_dim)
            weight_padded = F.pad(weight_f32, (0, pad_size))
            cols_padded = weight_padded.shape[1]
        else:
            weight_padded = weight_f32
            cols_padded = cols
            
        split_size = cols_padded // self.n_books
        weight_splits = []
        for i in range(self.n_books):
            split = weight_padded[:, i * split_size:(i + 1) * split_size]
            subvecs_per_row = split_size // self.codebook_dim
            split_reshaped = split.reshape(-1, self.codebook_dim)
            weight_splits.append(split_reshaped)
            
        all_indices = []
        quantized_splits = []
        avg_mse_list = []
        
        # 2. Оптимизация кодовых книг
        for book_idx in range(self.n_books):
            weight_split = weight_splits[book_idx]
            codebook = self.codebooks[book_idx]
            
            optimizer = torch.optim.Adam([codebook], lr=self.lr)

            # Инициализация индексов
            indices = self._find_best_indices(weight_split, codebook)
            best_loss = float('inf')
            best_indices = indices.clone()
            
            for iteration in range(num_iterations):
                optimizer.zero_grad()
                
                # ближайший вектор
                quantized_vectors_raw = codebook[indices] 
                
                # Straight-Through Estimator (STE) для обратного прохода
                # Градиент течет через codebook, обходя операцию argmin
                quantized_vectors_st = weight_split.detach() + (quantized_vectors_raw - quantized_vectors_raw.detach())
                
                # Reconstruction Loss - обучает codebook
                mse_loss = F.mse_loss(quantized_vectors_st, weight_split)
                
                # Commitment Loss - стабилизация
                commitment_loss = F.mse_loss(weight_split, quantized_vectors_raw.detach())
                total_loss = mse_loss + commitment_weight * commitment_loss
                
                total_loss.backward() 
                
                if mse_loss.item() < best_loss:
                    best_loss = mse_loss.item()
                    best_indices = indices.clone()
                
                with torch.no_grad():
                     distances = torch.cdist(weight_split, codebook)
                     indices = torch.argmin(distances, dim=1)
                
                optimizer.step()
            
            final_codebook = self.codebooks[book_idx]
            final_quantized_vectors = final_codebook[best_indices]
            
            all_indices.append(best_indices.to('cpu'))
            quantized_splits.append(final_quantized_vectors)
            avg_mse_list.append(best_loss)
            
        reconstructed_splits = []
        for i, quantized in enumerate(quantized_splits):
            split_size_original = cols_padded // self.n_books
            reconstructed = quantized.reshape(rows, split_size_original)
            reconstructed_splits.append(reconstructed)

        quantized_full = torch.cat(reconstructed_splits, dim=1)
        
        if pad_size > 0:
            quantized_full = quantized_full[:, :-pad_size]

        quantization_info = {
            "n_books": self.n_books,
            "codebook_size": self.codebook_size,
            "codebook_dim": self.codebook_dim,
            "compression_ratio": self._calculate_compression_ratio(weight, all_indices),
            "avg_mse": np.mean(avg_mse_list) if avg_mse_list else 0
        }
        
        return quantized_full.to(original_dtype), quantization_info
    
    def _calculate_compression_ratio(self, original_weight: torch.Tensor, indices: List[torch.Tensor]) -> float:
        # Логика расчета сжатия (без изменений)
        original_bits = original_weight.numel() * 16 
        codebook_bits = sum(cb.numel() * 16 for cb in self.codebooks)
        log2_k = np.ceil(np.log2(self.codebook_size))
        indices_bits = sum(idx.numel() * log2_k for idx in indices if idx is not None)
        compressed_bits = codebook_bits + indices_bits
        return original_bits / compressed_bits if compressed_bits > 0 else 1.0



#ГЛАВНЫЙ КОМПРЕССОР МОДЕЛИ 

class AdditiveQwenCompressor:
    
    def __init__(self, model_path="Qwen/Qwen3-8B", device=None):
        
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="cpu",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model.eval()
        
        self.stats = {"model": model_path, "layer_stats": {}, "global_stats": {}}

    def _transfer_layer_to_device(self, module: nn.Module) -> torch.Tensor:
        """Переносит веса модуля на GPU (self.device) для квантизации"""   
        weight = module.weight.data
        if weight.device.type != self.device:
            weight = weight.to(self.device)
            module.weight.data = weight
            
        if torch.cuda.is_available():
             torch.cuda.empty_cache() 
             
        return module.weight.data
        
    def _transfer_layer_to_cpu(self, module: nn.Module):
        """Возвращает веса модуля на CPU"""
        if module.weight.data.device.type == self.device:
             module.weight.data = module.weight.data.to("cpu")
             
        if torch.cuda.is_available():
             torch.cuda.empty_cache() 

    def additive_quantize_layer(self,
                                layer_name: str,
                                module: nn.Module,
                                config: Dict) -> Dict:
        """Применение аддитивной квантизации к одному слою"""
        weight = self._transfer_layer_to_device(module)
        rows, cols = weight.shape
        original_dtype = weight.dtype
        
        quantizer = AdditiveQuantizer(
            n_books=config["n_books"],
            codebook_size=config["codebook_size"],
            codebook_dim=config["codebook_dim"],
            learning_rate=0.01,
            device=self.device
        )
        
        quantized_weight_f32, quant_info = quantizer.quantize(
            weight,
            num_iterations=config.get("iterations", 100),
            commitment_weight=0.25
        )
        
        weight_f32 = weight.to(torch.float32)
        mse_error = F.mse_loss(quantized_weight_f32, weight_f32).item()
        relative_error = mse_error / (torch.var(weight_f32).item() + 1e-8)
        
        module.weight.data = quantized_weight_f32.to(original_dtype)
        
        self._transfer_layer_to_cpu(module) # Квантованные веса теперь на CPU
        del weight_f32
        del quantized_weight_f32
        del weight
        del quantizer
        
        if torch.cuda.is_available():
             torch.cuda.empty_cache() 

        layer_stats = {
            "original_shape": [rows, cols],
            "config": config,
            "mse_error": mse_error,
            "relative_error": relative_error,
            "compression_ratio": quant_info["compression_ratio"],
            "avg_quantizer_mse": quant_info["avg_mse"]
        }
        
        return layer_stats
    def smart_layer_configuration(self, layer_name: str, weight_shape: Tuple[int, int]) -> Dict | None:
        """Умный подбор конфигурации для каждого слоя."""
        rows, cols = weight_shape
        total_params = rows * cols
        
        if total_params < 100000: return None
        if "lm_head" in layer_name.lower():
            return {"n_books": 1, "codebook_size": 1024, "codebook_dim": 256, "iterations": 200}
        if "embed" in layer_name.lower():
            return {"n_books": 2, "codebook_size": 512, "codebook_dim": 256, "iterations": 150}
        if any(proj in layer_name.lower() for proj in ["q_proj", "k_proj", "v_proj", "o_proj"]):
            return {"n_books": 4, "codebook_size": 256, "codebook_dim": 128, "iterations": 100}
        if any(proj in layer_name.lower() for proj in ["gate_proj", "up_proj", "down_proj"]):
             return {"n_books": 4, "codebook_size": 128, "codebook_dim": 64, "iterations": 80}
        return {"n_books": 3, "codebook_size": 256, "codebook_dim": 64, "iterations": 100}

    def compress_model(self, strategy: str = "smart", min_layer_size: int = 100000):
        linear_layers = []
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Linear) and module.weight.numel() >= min_layer_size:
                linear_layers.append((name, module))
        compressed_layers = 0
        
        for name, module in tqdm(linear_layers, desc="Сжатие слоев", unit="layer"):
            rows, cols = module.weight.shape
            config = self.smart_layer_configuration(name, (rows, cols))
            if config is None: continue
            
            try:
                layer_stats = self.additive_quantize_layer(name, module, config)
                self.stats["layer_stats"][name] = layer_stats
                compressed_layers += 1
                
                print(f"  [DONE] {name}: Ratio={layer_stats['compression_ratio']:.2f}x, MSE={layer_stats['mse_error']:.6f}")
                
            except Exception as e:
                print(f"\n Ошибка при квантизации слоя {name}: {e}")
                continue
        
        if self.stats["layer_stats"]:
            ratios = [s["compression_ratio"] for s in self.stats["layer_stats"].values()]
            errors = [s["mse_error"] for s in self.stats["layer_stats"].values()]
            self.stats["global_stats"] = {
                "compressed_layers": compressed_layers,
                "avg_compression_ratio": np.mean(ratios),
                "avg_mse_error": np.mean(errors),
                "strategy": strategy
            }

        if self.stats["global_stats"]:
            print(f"Ср. Compression Ratio: {self.stats['global_stats']['avg_compression_ratio']:.2f}x")
            print(f"Ср. MSE Ошибка: {self.stats['global_stats']['avg_mse_error']:.6f}")
        
        return self.stats
    
    # def save_compressed_model(self, output_path: str = "./qwen-additive-quant"):
    #     output_dir = Path(output_path)
    #     output_dir.mkdir(parents=True, exist_ok=True)
        
    #     self.model.save_pretrained(output_dir, safe_serialization=True)
    #     self.tokenizer.save_pretrained(output_dir)
        
    #     stats_file = output_dir / "additive_quant_stats.json"
    #     with open(stats_file, "w", encoding="utf-8") as f:
    #         json.dump(self.stats, f, indent=2, ensure_ascii=False)
            
    #     print(f"Модель и статистика сохранены в {output_path}")
    #     return output_dir



    def save_compressed_model(self, output_path: str = "./qwen-additive-quant"):
        temp_dir = Path("./temp_save")
        temp_dir.mkdir(parents=True, exist_ok=True)
        
        try:
            self.model.save_pretrained(temp_dir, safe_serialization=True)
            self.tokenizer.save_pretrained(temp_dir)

            stats_file = temp_dir / "additive_quant_stats.json"
            with open(stats_file, "w", encoding="utf-8") as f:
                json.dump(self.stats, f, indent=2, ensure_ascii=False)

            zip_path = Path(output_path).with_suffix('.zip')
            
            with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED, 
                               compresslevel=9) as zipf:
                for file_path in temp_dir.rglob('*'):
                    if file_path.is_file():

                        arcname = file_path.relative_to(temp_dir)
                        zipf.write(file_path, arcname)

            shutil.rmtree(temp_dir)
            
            return zip_path
            
        except Exception as e:
            if temp_dir.exists():
                shutil.rmtree(temp_dir)
            raise e


if __name__ == '__main__':
    MODEL_PATH = "Qwen/Qwen3-8B" 
    OUTPUT_PATH = "./qwen3-08b-quantized"
    
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        compressor = AdditiveQwenCompressor(model_path=MODEL_PATH, device=DEVICE)
        stats = compressor.compress_model(strategy="smart")
        compressor.save_compressed_model(OUTPUT_PATH)
        
    except Exception as e:
        print(f"\nFATAL ERROR: Произошла критическая ошибка: {e}")


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


 Ошибка при квантизации слоя lm_head: CUDA out of memory. Tried to allocate 9.27 GiB. GPU 0 has a total capacity of 14.74 GiB of which 8.78 GiB is free. Process 36139 has 5.96 GiB memory in use. Of the allocated memory 5.83 GiB is allocated by PyTorch, and 8.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Ср. Compression Ratio: 89.75x
Ср. MSE Ошибка: 0.000665



Архивация:   4%|▎         | 1/28 [00:35<15:50, 35.20s/it]

   Обработано 1/28 файлов, 928.1 MB


Архивация: 100%|██████████| 28/28 [04:44<00:00, 10.17s/it]

Ошибка с файлом model-00001-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом model-00007-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом model-00013-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом model-00002-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом added_tokens.json: [Errno 28] No space left on device
Ошибка с файлом tokenizer_config.json: [Errno 28] No space left on device
Ошибка с файлом chat_template.jinja: [Errno 28] No space left on device
Ошибка с файлом model-00010-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом model-00009-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом model-00008-of-00017.safetensors: [Errno 28] No space left on device
Ошибка с файлом vocab.json: [Errno 28] No space left on device
Ошибка с файлом additive_quant_stats.json: [Errno 28] No space left on device
Ошибка с файлом model-00016-of-00017.safetensors